### Ingest data on Google Earth Engine (WRI bucket)

* Purpose of script: This notebook will upload the geotiff files from the Google Cloud Storage to the WRI/aqueduct earthengine bucket. 
* Author: Rutger Hofste
* Kernel used: python36
* Date created: 20170802

## Preparation

* Authorize earthengine by running in your terminal: `earthengine authenticate`  
* you need to have access to the WRI-Aquaduct (yep a Google employee made a typo) bucket to ingest the data. Rutger can grant access to write to this folder. 
* Have access to the Google Cloud Storage Bucker

Run in your terminal `gcloud config set project aqueduct30`

In [1]:
import subprocess
import datetime
import os
import time
from datetime import timedelta
import re
import pandas as pd

## Settings

In [2]:
GCS_BASE = "gs://aqueduct30_v01/Y2017M08D02_RH_Upload_to_GoogleCS_V01/"

In [3]:
EE_BASE = "projects/WRI-Aquaduct/PCRGlobWB20V05"

## Functions

In [4]:
def splitKey(key):
    # will yield the root file code and extension of a set of keys
    prefix, extension = key.split(".")
    fileName = prefix.split("/")[-1]
    parameter = fileName[:-12]
    month = fileName[-2:] #can also do this with regular expressions if you like
    year = fileName[-7:-3]
    identifier = fileName[-11:-8]
    outDict = {"fileName":fileName,"extension":extension,"parameter":parameter,"month":month,"year":year,"identifier":identifier}
    return outDict

def splitParameter(parameter):
    #values = parameter.split("_")
    values = re.split("_|-", parameter) #soilmoisture uses a hyphen instead of underscore between the years
    keys = ["geographic_range","temporal_range","indicator","temporal_resolution","units","spatial_resolution","temporal_range_min","temporal_range_max"]
    # ['global', 'historical', 'PDomWN', 'month', 'millionm3', '5min', '1960', '2014']
    outDict = dict(zip(keys, values))
    outDict["parameter"] = parameter
    return outDict


## Script

In [5]:
command = ("earthengine create folder %s") %EE_BASE

In [6]:
print(command)

earthengine create folder projects/WRI-Aquaduct/PCRGlobWB20V05


In [7]:
subprocess.check_output(command,shell=True)

'Asset projects/WRI-Aquaduct/PCRGlobWB20V05 already exists\n'

In [8]:
command = ("/opt/google-cloud-sdk/bin/gsutil ls %s") %(GCS_BASE)

In [9]:
keys = subprocess.check_output(command,shell=True)

In [10]:
keys = keys.decode('UTF-8').splitlines()

Removing first item from the list. The first item contains a folder without file name

In [11]:
keys2 = keys[1:]

In [12]:
df = pd.DataFrame()
i = 0
for key in keys2:
    i = i+1
    outDict = splitKey(key)
    df2 = pd.DataFrame(outDict,index=[i])
    df = df.append(df2)    

In [13]:
df.head()

,extension,fileName,identifier,month,parameter,year
1,tif,global_historical_PDomWN_month_millionm3_5min_...,000,01,global_historical_PDomWN_month_millionm3_5min_...,1960
2,tif,global_historical_PDomWN_month_millionm3_5min_...,001,02,global_historical_PDomWN_month_millionm3_5min_...,1960
3,tif,global_historical_PDomWN_month_millionm3_5min_...,002,03,global_historical_PDomWN_month_millionm3_5min_...,1960
4,tif,global_historical_PDomWN_month_millionm3_5min_...,003,04,global_historical_PDomWN_month_millionm3_5min_...,1960
5,tif,global_historical_PDomWN_month_millionm3_5min_...,004,05,global_historical_PDomWN_month_millionm3_5min_...,1960


In [14]:
df.tail()

,extension,fileName,identifier,month,parameter,year
9286,tif,global_historical_soilmoisture_month_meter_5mi...,679,08,global_historical_soilmoisture_month_meter_5mi...,2014
9287,tif,global_historical_soilmoisture_month_meter_5mi...,680,09,global_historical_soilmoisture_month_meter_5mi...,2014
9288,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014
9289,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014
9290,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014


In [15]:
df.shape

(9290, 6)

In [16]:
parameters = df.parameter.unique()

In [17]:
print(parameters)

[u'global_historical_PDomWN_month_millionm3_5min_1960_2014'
 u'global_historical_PDomWN_year_millionm3_5min_1960_2014'
 u'global_historical_PDomWW_month_millionm3_5min_1960_2014'
 u'global_historical_PDomWW_year_millionm3_5min_1960_2014'
 u'global_historical_PIndWN_month_millionm3_5min_1960_2014'
 u'global_historical_PIndWN_year_millionm3_5min_1960_2014'
 u'global_historical_PIndWW_month_millionm3_5min_1960_2014'
 u'global_historical_PIndWW_year_millionm3_5min_1960_2014'
 u'global_historical_PIrrWN_month_millionm3_5min_1960_2014'
 u'global_historical_PIrrWN_year_millionm3_5min_1960_2014'
 u'global_historical_PIrrWW_month_millionm3_5min_1960_2014'
 u'global_historical_PIrrWW_year_millionm3_5min_1960_2014'
 u'global_historical_PLivWN_month_millionm3_5min_1960_2014'
 u'global_historical_PLivWN_year_millionm3_5min_1960_2014'
 u'global_historical_PLivWW_month_millionm3_5min_1960_2014'
 u'global_historical_PLivWW_year_millionm3_5min_1960_2014'
 u'global_historical_aqbasinwaterstress_month_di

We will store the geotiff images of each NetCDF4 file in imageCollections. The imageCollections will have the same name and content as the original NetCDF4files. 


In [18]:
for parameter in parameters:
    eeLocation = EE_BASE + "/" + parameter
    command = ("earthengine create collection %s") %eeLocation
    # Uncomment the following command if you run this script for the first time
    #subprocess.check_output(command,shell=True)
    #print(command)
    

Now that the folder and collections have been created we can start ingesting the data. It is crucial to store the relevant metadata with the images. 

In [19]:
df_parameter = pd.DataFrame()
i = 0
for parameter in parameters:
    i = i+1
    outDict_parameter = splitParameter(parameter)
    df_parameter2 = pd.DataFrame(outDict_parameter,index=[i])
    df_parameter = df_parameter.append(df_parameter2)   
    

In [20]:
df_parameter.head()

,geographic_range,indicator,parameter,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units
1,global,PDomWN,global_historical_PDomWN_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3
2,global,PDomWN,global_historical_PDomWN_year_millionm3_5min_1...,5min,historical,2014,1960,year,millionm3
3,global,PDomWW,global_historical_PDomWW_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3
4,global,PDomWW,global_historical_PDomWW_year_millionm3_5min_1...,5min,historical,2014,1960,year,millionm3
5,global,PIndWN,global_historical_PIndWN_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3


In [21]:
df_parameter.tail()

,geographic_range,indicator,parameter,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units
21,global,riverdischarge,global_historical_riverdischarge_month_m3secon...,5min,historical,2014,1960,month,m3second
22,global,riverdischarge,global_historical_riverdischarge_year_m3second...,5min,historical,2014,1960,year,m3second
23,global,runoff,global_historical_runoff_month_mmonth_5min_195...,5min,historical,2014,1958,month,mmonth
24,global,runoff,global_historical_runoff_year_myear_5min_1958_...,5min,historical,2014,1958,year,myear
25,global,soilmoisture,global_historical_soilmoisture_month_meter_5mi...,5min,historical,2014,1958,month,meter


In [22]:
df_parameter.shape

(25, 9)

In [23]:
df_complete = df.merge(df_parameter,how='left',left_on='parameter',right_on='parameter')

Adding NoData value, ingested_by and exportdescription

In [24]:
df_complete["nodata"] = -9999
df_complete["ingested_by"] ="RutgerHofste"
df_complete["exportdescription"] = df_complete["indicator"] + "_" + df_complete["temporal_resolution"]+"Y"+df_complete["year"]+"M"+df_complete["month"]

In [25]:
df_complete.head()

,extension,fileName,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata,ingested_by,exportdescription
0,tif,global_historical_PDomWN_month_millionm3_5min_...,000,01,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M01
1,tif,global_historical_PDomWN_month_millionm3_5min_...,001,02,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M02
2,tif,global_historical_PDomWN_month_millionm3_5min_...,002,03,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M03
3,tif,global_historical_PDomWN_month_millionm3_5min_...,003,04,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M04
4,tif,global_historical_PDomWN_month_millionm3_5min_...,004,05,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M05


In [26]:
df_complete.tail()

,extension,fileName,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata,ingested_by,exportdescription
9285,tif,global_historical_soilmoisture_month_meter_5mi...,679,08,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M08
9286,tif,global_historical_soilmoisture_month_meter_5mi...,680,09,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M09
9287,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M10
9288,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M11
9289,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M12


In [27]:
list(df_complete.columns.values)

['extension',
 'fileName',
 'identifier',
 'month',
 'parameter',
 'year',
 'geographic_range',
 'indicator',
 'spatial_resolution',
 'temporal_range',
 'temporal_range_max',
 'temporal_range_min',
 'temporal_resolution',
 'units',
 'nodata',
 'ingested_by',
 'exportdescription']

In [30]:
def uploadEE(index,row):
    target = EE_BASE +"/"+ row.parameter + "/" + row.fileName
    source = GCS_BASE + row.fileName + "." + row.extension
    metadata = "--nodata_value=%s --time_start %s-%s-01 -p extension=%s -p filename=%s -p identifier=%s -p year=%s -p geographic_range=%s -p indicator=%s -p spatial_resolution=%s -p temporal_range=%s -p temporal_range_max=%s -p temporal_range_min=%s -p temporal_resolution=%s -p units=%s -p ingested_by=%s -p exportdescription=%s" %(row.nodata,row.year,row.month,row.extension,row.fileName,row.identifier,row.year,row.geographic_range,row.indicator,row.spatial_resolution,row.temporal_range,row.temporal_range_max,row.temporal_range_min, row.temporal_resolution, row.units, row.ingested_by, row.exportdescription)
    command = "/opt/anaconda3/bin/earthengine upload image --asset_id %s %s %s" % (target, source,metadata)
    try:
        response = subprocess.check_output(command, shell=True)
        outDict = {"command":command,"response":response,"error":0}
        df_errors2 = pd.DataFrame(outDict,index=[index])
        pass
    except:
        try:
            outDict = {"command":command,"response":response,"error":1}
        except:
            outDict = {"command":command,"response":-9999,"error":2}
        df_errors2 = pd.DataFrame(outDict,index=[index])
        print("error")
    return df_errors2



In [ ]:
df_errors = pd.DataFrame()
start_time = time.time()
for index, row in df_complete.iterrows():
    elapsed_time = time.time() - start_time 
    print(index,"%.2f" %((index/8548.0)*100), "elapsed: ", str(timedelta(seconds=elapsed_time)))
    df_errors2 = uploadEE(index,row)
    df_errors = df_errors.append(df_errors2)
    
    

(0, '0.00', 'elapsed: ', '0:00:00.012130')
(1, '0.01', 'elapsed: ', '0:00:01.442132')
(2, '0.02', 'elapsed: ', '0:00:02.606816')
(3, '0.04', 'elapsed: ', '0:00:03.816287')
(4, '0.05', 'elapsed: ', '0:00:04.980202')
(5, '0.06', 'elapsed: ', '0:00:06.155003')
(6, '0.07', 'elapsed: ', '0:00:07.309283')
(7, '0.08', 'elapsed: ', '0:00:08.442458')
(8, '0.09', 'elapsed: ', '0:00:09.617328')
(9, '0.11', 'elapsed: ', '0:00:10.757391')
(10, '0.12', 'elapsed: ', '0:00:12.015491')
(11, '0.13', 'elapsed: ', '0:00:13.226300')
(12, '0.14', 'elapsed: ', '0:00:14.428539')
(13, '0.15', 'elapsed: ', '0:00:15.695246')
(14, '0.16', 'elapsed: ', '0:00:16.936033')
(15, '0.18', 'elapsed: ', '0:00:18.063282')
(16, '0.19', 'elapsed: ', '0:00:19.234435')
(17, '0.20', 'elapsed: ', '0:00:20.416357')
(18, '0.21', 'elapsed: ', '0:00:21.583643')
(19, '0.22', 'elapsed: ', '0:00:22.793715')
(20, '0.23', 'elapsed: ', '0:00:23.908967')
(21, '0.25', 'elapsed: ', '0:00:25.036050')
(22, '0.26', 'elapsed: ', '0:00:26.247714'

(185, '2.16', 'elapsed: ', '0:03:45.831343')
(186, '2.18', 'elapsed: ', '0:03:46.922072')
(187, '2.19', 'elapsed: ', '0:03:47.991843')
(188, '2.20', 'elapsed: ', '0:03:49.162332')
(189, '2.21', 'elapsed: ', '0:03:50.338605')
(190, '2.22', 'elapsed: ', '0:03:51.465543')
(191, '2.23', 'elapsed: ', '0:03:52.602319')
(192, '2.25', 'elapsed: ', '0:03:53.784525')
(193, '2.26', 'elapsed: ', '0:03:55.018508')
(194, '2.27', 'elapsed: ', '0:03:56.088998')
(195, '2.28', 'elapsed: ', '0:03:57.198106')
(196, '2.29', 'elapsed: ', '0:03:58.368086')
(197, '2.30', 'elapsed: ', '0:03:59.545212')
(198, '2.32', 'elapsed: ', '0:04:00.792572')
(199, '2.33', 'elapsed: ', '0:04:02.054067')
(200, '2.34', 'elapsed: ', '0:04:03.193201')
(201, '2.35', 'elapsed: ', '0:04:04.360057')
(202, '2.36', 'elapsed: ', '0:04:05.521603')
(203, '2.37', 'elapsed: ', '0:04:06.662887')
(204, '2.39', 'elapsed: ', '0:04:07.783256')
(205, '2.40', 'elapsed: ', '0:04:08.970049')
(206, '2.41', 'elapsed: ', '0:04:10.093355')
(207, '2.4

(368, '4.31', 'elapsed: ', '0:07:26.537960')
(369, '4.32', 'elapsed: ', '0:07:27.630547')
(370, '4.33', 'elapsed: ', '0:07:28.794948')
(371, '4.34', 'elapsed: ', '0:07:29.999826')
(372, '4.35', 'elapsed: ', '0:07:31.134382')
(373, '4.36', 'elapsed: ', '0:07:32.264391')
(374, '4.38', 'elapsed: ', '0:07:33.340269')
(375, '4.39', 'elapsed: ', '0:07:34.542902')
(376, '4.40', 'elapsed: ', '0:07:35.648335')
(377, '4.41', 'elapsed: ', '0:07:37.036933')
(378, '4.42', 'elapsed: ', '0:07:38.156094')
(379, '4.43', 'elapsed: ', '0:07:39.236006')
(380, '4.45', 'elapsed: ', '0:07:40.891491')
(381, '4.46', 'elapsed: ', '0:07:42.030798')
(382, '4.47', 'elapsed: ', '0:07:43.189106')
(383, '4.48', 'elapsed: ', '0:07:44.400879')
(384, '4.49', 'elapsed: ', '0:07:45.512351')
(385, '4.50', 'elapsed: ', '0:07:46.696253')
(386, '4.52', 'elapsed: ', '0:07:47.799867')
(387, '4.53', 'elapsed: ', '0:07:48.893019')
(388, '4.54', 'elapsed: ', '0:07:50.037887')
(389, '4.55', 'elapsed: ', '0:07:51.166215')
(390, '4.5

(551, '6.45', 'elapsed: ', '0:11:11.444089')
(552, '6.46', 'elapsed: ', '0:11:12.583400')
(553, '6.47', 'elapsed: ', '0:11:13.647137')
(554, '6.48', 'elapsed: ', '0:11:19.516206')
(555, '6.49', 'elapsed: ', '0:11:20.620609')
(556, '6.50', 'elapsed: ', '0:11:21.705802')
(557, '6.52', 'elapsed: ', '0:11:22.801057')
(558, '6.53', 'elapsed: ', '0:11:25.154837')
(559, '6.54', 'elapsed: ', '0:11:26.314520')
(560, '6.55', 'elapsed: ', '0:11:27.359964')
(561, '6.56', 'elapsed: ', '0:11:28.430975')
(562, '6.57', 'elapsed: ', '0:11:29.621970')
(563, '6.59', 'elapsed: ', '0:11:30.712526')
(564, '6.60', 'elapsed: ', '0:11:31.824231')
(565, '6.61', 'elapsed: ', '0:11:32.983212')
(566, '6.62', 'elapsed: ', '0:11:34.096585')
(567, '6.63', 'elapsed: ', '0:11:35.334800')
(568, '6.64', 'elapsed: ', '0:11:36.463699')
(569, '6.66', 'elapsed: ', '0:11:37.595281')
(570, '6.67', 'elapsed: ', '0:11:38.646547')
(571, '6.68', 'elapsed: ', '0:11:39.795037')
(572, '6.69', 'elapsed: ', '0:11:40.969971')
(573, '6.7

(734, '8.59', 'elapsed: ', '0:15:17.537032')
(735, '8.60', 'elapsed: ', '0:15:18.638619')
(736, '8.61', 'elapsed: ', '0:15:19.766131')
(737, '8.62', 'elapsed: ', '0:15:20.949114')
(738, '8.63', 'elapsed: ', '0:15:22.043836')
(739, '8.65', 'elapsed: ', '0:15:23.149270')
(740, '8.66', 'elapsed: ', '0:15:24.279311')
(741, '8.67', 'elapsed: ', '0:15:25.469113')
(742, '8.68', 'elapsed: ', '0:15:26.428702')
(743, '8.69', 'elapsed: ', '0:15:27.714849')
(744, '8.70', 'elapsed: ', '0:15:28.855535')
(745, '8.72', 'elapsed: ', '0:15:29.974919')
(746, '8.73', 'elapsed: ', '0:15:31.102997')
(747, '8.74', 'elapsed: ', '0:15:32.348186')
(748, '8.75', 'elapsed: ', '0:15:34.731867')
(749, '8.76', 'elapsed: ', '0:15:35.885501')
(750, '8.77', 'elapsed: ', '0:15:36.964874')
(751, '8.79', 'elapsed: ', '0:15:38.316739')
(752, '8.80', 'elapsed: ', '0:15:39.476741')
(753, '8.81', 'elapsed: ', '0:15:40.598982')
(754, '8.82', 'elapsed: ', '0:15:41.760050')
(755, '8.83', 'elapsed: ', '0:15:42.885759')
(756, '8.8

...

(2317, '27.11', 'elapsed: ', '0:47:28.667773')
(2318, '27.12', 'elapsed: ', '0:47:30.089279')
(2319, '27.13', 'elapsed: ', '0:47:31.254441')
(2320, '27.14', 'elapsed: ', '0:47:32.265194')
(2321, '27.15', 'elapsed: ', '0:47:33.344265')
(2322, '27.16', 'elapsed: ', '0:47:34.437889')
(2323, '27.18', 'elapsed: ', '0:47:35.552713')
(2324, '27.19', 'elapsed: ', '0:47:36.681122')
(2325, '27.20', 'elapsed: ', '0:47:37.785712')
(2326, '27.21', 'elapsed: ', '0:47:38.912238')
(2327, '27.22', 'elapsed: ', '0:47:40.106790')
(2328, '27.23', 'elapsed: ', '0:47:41.242642')
(2329, '27.25', 'elapsed: ', '0:47:42.468034')
(2330, '27.26', 'elapsed: ', '0:47:43.615599')
(2331, '27.27', 'elapsed: ', '0:47:44.757854')
(2332, '27.28', 'elapsed: ', '0:47:45.899227')
(2333, '27.29', 'elapsed: ', '0:47:46.950597')
(2334, '27.30', 'elapsed: ', '0:47:48.090081')
(2335, '27.32', 'elapsed: ', '0:47:49.319158')
(2336, '27.33', 'elapsed: ', '0:47:50.535601')
(2337, '27.34', 'elapsed: ', '0:47:51.689716')
(2338, '27.35

(2492, '29.15', 'elapsed: ', '0:50:54.502907')
(2493, '29.16', 'elapsed: ', '0:50:55.483504')
(2494, '29.18', 'elapsed: ', '0:50:56.607200')
(2495, '29.19', 'elapsed: ', '0:50:57.742701')
(2496, '29.20', 'elapsed: ', '0:50:59.180933')
(2497, '29.21', 'elapsed: ', '0:51:00.371614')
(2498, '29.22', 'elapsed: ', '0:51:01.538800')
(2499, '29.23', 'elapsed: ', '0:51:02.734155')
(2500, '29.25', 'elapsed: ', '0:51:03.977621')
(2501, '29.26', 'elapsed: ', '0:51:05.150561')
(2502, '29.27', 'elapsed: ', '0:51:06.311744')
(2503, '29.28', 'elapsed: ', '0:51:07.824796')
(2504, '29.29', 'elapsed: ', '0:51:08.961391')
(2505, '29.31', 'elapsed: ', '0:51:10.043366')
(2506, '29.32', 'elapsed: ', '0:51:11.176385')
(2507, '29.33', 'elapsed: ', '0:51:12.361917')
(2508, '29.34', 'elapsed: ', '0:51:13.494434')
(2509, '29.35', 'elapsed: ', '0:51:14.778441')
(2510, '29.36', 'elapsed: ', '0:51:15.947130')
(2511, '29.38', 'elapsed: ', '0:51:17.149135')
(2512, '29.39', 'elapsed: ', '0:51:19.384097')
(2513, '29.40

(2667, '31.20', 'elapsed: ', '0:54:42.012780')
(2668, '31.21', 'elapsed: ', '0:54:43.117469')
(2669, '31.22', 'elapsed: ', '0:54:44.285846')
(2670, '31.24', 'elapsed: ', '0:54:45.471994')
(2671, '31.25', 'elapsed: ', '0:54:46.633605')
(2672, '31.26', 'elapsed: ', '0:54:47.751062')
(2673, '31.27', 'elapsed: ', '0:54:49.053752')
(2674, '31.28', 'elapsed: ', '0:54:50.119106')
(2675, '31.29', 'elapsed: ', '0:54:51.273506')
(2676, '31.31', 'elapsed: ', '0:54:52.684678')
(2677, '31.32', 'elapsed: ', '0:54:53.997800')
(2678, '31.33', 'elapsed: ', '0:54:55.167245')
(2679, '31.34', 'elapsed: ', '0:54:56.332576')
(2680, '31.35', 'elapsed: ', '0:54:57.568760')
(2681, '31.36', 'elapsed: ', '0:54:58.720736')
(2682, '31.38', 'elapsed: ', '0:54:59.982418')
(2683, '31.39', 'elapsed: ', '0:55:08.487494')
(2684, '31.40', 'elapsed: ', '0:55:09.755384')
(2685, '31.41', 'elapsed: ', '0:55:10.888119')
(2686, '31.42', 'elapsed: ', '0:55:12.005338')
(2687, '31.43', 'elapsed: ', '0:55:14.722969')
(2688, '31.45

(2842, '33.25', 'elapsed: ', '0:58:23.918438')
(2843, '33.26', 'elapsed: ', '0:58:24.980141')
(2844, '33.27', 'elapsed: ', '0:58:26.139625')
(2845, '33.28', 'elapsed: ', '0:58:27.328870')
(2846, '33.29', 'elapsed: ', '0:58:28.526467')
(2847, '33.31', 'elapsed: ', '0:58:37.462948')
(2848, '33.32', 'elapsed: ', '0:58:38.585410')
(2849, '33.33', 'elapsed: ', '0:58:39.792744')
(2850, '33.34', 'elapsed: ', '0:58:41.015927')
(2851, '33.35', 'elapsed: ', '0:58:42.172012')
(2852, '33.36', 'elapsed: ', '0:58:43.266617')
(2853, '33.38', 'elapsed: ', '0:58:44.429149')
(2854, '33.39', 'elapsed: ', '0:58:45.728406')
(2855, '33.40', 'elapsed: ', '0:58:46.841681')
(2856, '33.41', 'elapsed: ', '0:58:48.055732')
(2857, '33.42', 'elapsed: ', '0:58:49.300390')
(2858, '33.43', 'elapsed: ', '0:58:50.396061')
(2859, '33.45', 'elapsed: ', '0:58:51.744614')
(2860, '33.46', 'elapsed: ', '0:58:52.863515')
(2861, '33.47', 'elapsed: ', '0:58:54.036254')
(2862, '33.48', 'elapsed: ', '0:58:55.284256')
(2863, '33.49

(3017, '35.29', 'elapsed: ', '1:02:20.329894')
(3018, '35.31', 'elapsed: ', '1:02:21.487706')
(3019, '35.32', 'elapsed: ', '1:02:22.599609')
(3020, '35.33', 'elapsed: ', '1:02:23.761077')
(3021, '35.34', 'elapsed: ', '1:02:24.872998')
(3022, '35.35', 'elapsed: ', '1:02:26.146349')


In [ ]:
df_errors.head()

In [ ]:
!mkdir /volumes/data/temp

In [ ]:
df_errors.to_csv("/volumes/data/temp/df_errors.csv")

In [ ]:
!aws s3 cp  /volumes/data/temp/df_errors.csv s3://wri-projects/Aqueduct30/temp/df_errors.csv